## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBSBSBSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
953
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|2840.8720295402313|-1025.223109780797|-345.8072054341986|    412.1|BBSBSBSB| 47.3864| -92.8389|       SNWD|0.23536631597571686|0.13578293696097948|0.12445324347418106|  0.2709215925

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [19]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
         'res_1', 'res_2', 'res_3', 'res_mean', 'year', 'station'])
# Features='station, year, coeff_1, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT coeff_1, coeff_2, coeff_3, elevation, latitude, longitude, res_1, res_2, res_3, res_mean, year, station FROM weather


,coeff_1,coeff_2,coeff_3,elevation,latitude,longitude,res_1,res_2,res_3,res_mean,year,station
0,2840.872030,-1025.223110,-345.807205,412.1,47.3864,-92.8389,0.235366,0.135783,0.124453,0.270922,1989.0,USW00094931
1,2514.263170,783.458386,438.246712,396.8,47.2467,-94.2228,0.223521,0.148126,0.124535,0.232054,1996.0,USC00214652
2,2255.808674,-879.984282,-739.568878,360.6,48.5614,-93.3981,0.305085,0.199336,0.124642,0.226308,1989.0,USW00014918
3,6544.970494,-26.949997,880.716217,438.9,47.5000,-92.5500,0.140593,0.140578,0.125017,0.499372,1975.0,USC00218543
4,3224.607507,342.332914,-1719.791201,438.9,47.5000,-92.5500,0.325035,0.317427,0.125437,0.306469,1969.0,USC00218543


In [20]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,CA006020559,CA006025203,CA006025205,CA006032119,CA006034075,CA006036904,USC00210059,USC00210515,USC00210520,USC00210643,...,USC00217460,USC00218543,USC00218618,USC00218621,USC00218700,USC00218763,USC00219059,USC00219173,USW00014918,USW00094931
year,,,,,,,,,,,,,,,,,,,,,
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1203.427508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.842486,NaN
1949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2368.961304,...,NaN,NaN,-1144.086718,NaN,NaN,2795.633656,804.176940,NaN,1206.196327,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1294.208751,...,490.666957,NaN,-288.480337,NaN,NaN,NaN,1071.086092,NaN,1733.661895,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,778.798448,NaN,2671.013253,...,113.286983,NaN,980.409387,NaN,NaN,1226.800188,1563.422650,NaN,997.927081,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,NaN,972.257645,NaN,NaN,-25.444379,...,181.857826,NaN,-759.160133,NaN,NaN,-978.652097,58.644408,NaN,-780.110144,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,7.070916,-1620.057015,NaN,-1083.864946,...,-91.375774,NaN,-1685.836506,NaN,NaN,-215.981525,-1189.397059,NaN,-444.240636,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,46.506181,-428.589500,NaN,495.634125,...,-241.710956,NaN,778.177511,NaN,NaN,446.533431,1729.336562,NaN,853.656110,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,307.273121,1007.803004,NaN,816.598897,...,544.812354,NaN,-101.959989,NaN,NaN,1233.761020,316.718596,NaN,919.154186,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,1648.183775,2189.257088,NaN,1881.908626,...,1232.836862,NaN,1048.665995,NaN,NaN,3541.918337,2165.199657,NaN,1487.928817,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [21]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

def frac(a,b):
    return ((a-b)*100.0)/a

print '** coeff_1 **'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ', str(RMS(tbl_minus_station)) + ', fraction explained=', frac (RMS(year_station_table), RMS(tbl_minus_station))
print 'RMS removing mean-by-year   = ', str(RMS(tbl_minus_year)) + ', fraction explained=', frac (RMS(year_station_table), RMS(tbl_minus_year))


** coeff_1 **
total RMS                   =  1607.08350635
RMS removing mean-by-station=  1475.61439358, fraction explained= 8.18060245454
RMS removing mean-by-year   =  852.75670797, fraction explained= 46.9376230543


In [14]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1607.08350635
0 after removing mean by year    = 852.75670797
0 after removing mean by stations= 720.349998818
1 after removing mean by year    = 711.670589287
1 after removing mean by stations= 710.589084345
2 after removing mean by year    = 710.36515461
2 after removing mean by stations= 710.292296096
3 after removing mean by year    = 710.260817294
3 after removing mean by stations= 710.245225063
4 after removing mean by year    = 710.237015693
4 after removing mean by stations= 710.23257353
